In [4]:
# %load ../firstcell.py
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
from lightning.pytorch import LightningModule
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer


# BERT Model definition
class GLUETransformer(LightningModule):
    def __init__(self):
        self.config = AutoConfig.from_pretrained(
            model_name_or_path, num_labels=num_labels
        )
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name_or_path, config=self.config
        )

    def forward(self, **inputs):
        return self.model(**inputs)


model = GLUETransformer(model_name_or_path="Intel/bert-base-uncased-mrpc")

In [ ]:
from neural_compressor.config import (
    PostTrainingQuantConfig,
    TuningCriterion,
    AccuracyCriterion,
)

accuracy_criterion = AccuracyCriterion(tolerable_loss=0.01)
tuning_criterion = TuningCriterion(max_trials=600)
conf = PostTrainingQuantConfig(
    approach="static",
    backend="default",
    tuning_criterion=tuning_criterion,
    accuracy_criterion=accuracy_criterion,
)

In [ ]:
from neural_compressor.quantization import fit

q_model = fit(
    model=model.model, conf=conf, calib_dataloader=val_dataloader(), eval_func=eval_func
)

In [ ]:
q_model.save("./saved_model/")